# DASH - BOOTSTRAP VERSION

In [1]:
pip install dash dash-bootstrap-components scikit-learn plotly pandas

Note: you may need to restart the kernel to use updated packages.


## Imports

In [7]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier

## Load Dataset

In [13]:
df = pd.read_csv('cancer_ml_prediction.csv')

In [15]:
feature_list = df.drop('class',axis=1).columns.tolist()
feature_list

['thickness',
 'size',
 'shape',
 'adhesion',
 'single',
 'nuclei',
 'chromatin',
 'nucleoli']

In [17]:
target_col = 'class'

## PCA (2 Components)

In [20]:
pca = PCA(n_components=2)
df['PC1'],df['PC2'] = pca.fit_transform(df[feature_list]).T

## RandomForest Feature Importances

In [23]:
rf = RandomForestClassifier().fit(df[feature_list],df[target_col])
importances = pd.DataFrame({'feature':feature_list,
                            'importance':rf.feature_importances_
                           }).sort_values(by='importance', ascending=False)

## DASH App

In [34]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([

    dbc.Row([
        dbc.Col(html.H2("Cancer Diagnosis Dashboard"), width=12)
    ], className="my-3"),

    dbc.Row([
        dbc.Col([
            html.Label("Select Feature"),
            dcc.Dropdown(
                id="feature-dropdown",
                options=[{"label": f, "value": f} for f in feature_list],
                value="shape",
                clearable=False
            )
        ], width=4)
    ]),

    dbc.Row([
        dbc.Col(dcc.Graph(id="pca-plot"), width=6),
        dbc.Col(dcc.Graph(id="importance-plot"), width=6),
    ]),

    dbc.Row([
        dbc.Col(dcc.Graph(id="violin-plot"), width=6),
        dbc.Col(dcc.Graph(id="boxplot"), width=6),
    ]),

    dbc.Row([
        dbc.Col(dcc.Graph(id="histogram"), width=12),
    ])

], fluid=True)

# CALLBACK
@app.callback(
    Output("violin-plot", "figure"),
    Output("boxplot", "figure"),
    Output("histogram", "figure"),
    Output("pca-plot", "figure"),
    Output("importance-plot", "figure"),
    Input("feature-dropdown", "value")
)
def update_plots(feature):

    # Violin plot
    violin_fig = px.violin(df, x=target_col, y=feature, color=target_col,
                           box=True, points="all")

    # Boxplot
    box_fig = px.box(df, x=target_col, y=feature, color=target_col)

    # Histogram
    hist_fig = px.histogram(df, x=feature, color=target_col, barmode="overlay")

    # PCA Scatter
    pca_fig = px.scatter(df, x="PC1", y="PC2", color=target_col,
                         title="PCA Scatter Plot")

    # Feature Importance Bar Chart
    importance_fig = px.bar(
        importances,
        x="importance", y="feature",
        orientation="h",
        title="Feature Importances"
    )

    return violin_fig, box_fig, hist_fig, pca_fig, importance_fig


if __name__ == "__main__":
    app.run(debug=True)